# Analiza danych w języku Python - ćwiczenia laboratoryjne 2024/2025

Ten notatnik zalicza się do grupy zestawów zadań, na podstawie których odbywa się zaliczenie ćwiczeń i podlega zwrotowi do oceny w ustalonym na zajęciach terminie.

Uwagi i wytyczne ogólne dotyczące uzupełniania i oceny notatnika:
- Podczas wykonywania zadań należy korzystać wyłącznie z pakietów zaimportowanych na początku notatnika oraz z pakietów wchodzących w skład standardowej biblioteki Pythona, które można zaimportować samodzielnie we wskazanej komórce notatnika. Wyjątek stanowi ostatni z projektów, gdzie po konsultacji z prowadzącym zajęcia możliwa jest większa swoboda w doborze pakietów.
- Swoje rozwiązania należy wprowadzać wyłącznie w miejce następujących fragmentów kodu:<br/> `# YOUR CODE HERE`<br/> `raise NotImplementedError()`<br/> 
a odpowiedzi tekstowe w komórkach oznaczonych hasłem:<br/> 
`YOUR ANSWER HERE`<br/> 
Nie należy w żaden sposób modyfikować pozostałych fragmentów kodu oraz innych elementów notatnika, w szczególności dodawać lub usuwać komórek oraz zmieniać nazwy pliku.
- Jeżeli zestaw zadań wymaga skorzystania z fragmentów kodu opracowanego w ramach wcześniejszych zestawów zadań należy je umieścić we wskazanej komórce notatnika.
- Otrzymywane wyniki i odpowiedzi mają być rezultatem wykonania napisanego kodu, odpowiedzi uzupełniane manualnie nie podlegają ocenie.
- Zawarte w notatniku automatyczne testy mają charakter poglądowy. Dotyczą one wybranych aspektów zadań i mają na celu wyłapać podstawowe błędy. Przejście przez kod wszystkich testów nie oznacza, że zadanie jest wykonane w całości poprawnie i zostanie ocenione na maksymalną liczbę punktów.
- Zadania należy wykonać w taki sposób, aby podczas wykonywania kodu nie zostały wyświetlone żadne ostrzeżenia.
- Zadania, które powodują wyświetlenie komunikatu o błędzie przerywającym wykonywanie kodu nie podlegają ocenie.

Uwagi i wytyczne ogólne dotyczące wizualizacji wyników:
- Wszystkie wykresy powinny być wykonane w jednolitym, przejrzystym i czytelnym stylu, posiadać odpowiednio dobrane proporcje i zakresy wartości osi.
- Figury powinny mieć ustawione białe tło, tak, aby niezależnie od ustawień notatnika wszystkie elementy wykresów były dobrze widoczne (domyślnie tło jest przeźroczyste co może powodować problemy w notatnikach z ustawionym ciemnym tłem). Rozmiar poziomy figur nie powinien przekraczać 20 cali.
- Wykresy oraz ich osie powinny mieć nadane tytuły. Jeżeli w obrębie figury znajduje się więcej niż jeden wykres to figura również powinna mieć nadany tytuł.

Przed odesłaniem zestawu zadań do oceny proszę uzupełnić komórkę z danymi autorów rozwiązania (nazwa zespołu oraz imiona, nazwiska i numery indeksów członków zespołu) oraz upewnić się, że notatnik działa zgodnie z oczekiwaniami. W tym celu należy skorzystać z opcji **Restart Kernel and Run All Cells...** dostępnej na górnej belce notatnika pod symbolem $\blacktriangleright\blacktriangleright$.

Nazwa zespołu:
Członkowie:

---

## Zestaw zadań 1: Wulkanizm w holocenie (Notatnik 1/3)

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Miejsce do importu pakietów wchodzących w skład standardowej biblioteki Pythona oraz ustawienie opcji wykorzystywanych pakietów

### Zadanie 1: Przygotowanie danych [10 pkt]

Plik raw_data.csv zawiera informacje na temat erupcji wulkanów pobrane ze strony https://volcano.si.edu/.

Zaimportuj zawarte w pliku dane do notatnika a następnie utwórz na ich podstawie tablicę `eruptions_data` wykonaną według następujących wytycznych:

- Tablica powinna zawierać wyłącznie następujące elementy:</br>

    > Indeks: Eruption Number</br>
   
    > Kolumny: Volcano Name, Eruption Category, VEI, Start Year, Start Month, Start Day, End Year, End Month, End Day, Evidence Type, Evidence Method</br>

- Indeks tablicy tworzony jest w oparciu o kolumnę Eruption Number z wczytanej tablicy danych.
- Kolumny Evidence Type i Evidence Method powstają poprzez rozbicie kolumny Evidence Method (dating) zawartej w danych wejściowych na dwie osobne kolumny. Zamień brakujące wartości w tych kolumnach (NaN) odpowiednio na 'Uncertain' i 'Unspecified'.
- Tablica nie powinna zawierać wierszy dotyczących podważanych (ang. discredited) erupcji oraz erupcji, dla których nie została określona wartość VEI (Volcanic Explosivity Index).
- Z nazw zawartych w kolumnie Eruption Category należy usunąć słowo "Eruption" pozostawiając tylko poprzedzające go człony zawartych w niej określeń.
- Niektóre z erupcji mają wpisane wartości miesiąca i dnia ich początku lub końca jako 0, należy zmienić te wartości na NaN.

Zapisz tablicę do pliku eruption_data.pkl.

Wyświetl przygotowaną tablicę w taki sposób aby widoczne było po 10 pierwszych i ostatnich wierszy, a zawarte w tablicy liczby wyświetlały się bez miejsc po przecinku (przydatne funkcje: `pandas.io.formats.style.Styler.format_index()`, `pandas.io.formats.style.Styler.format()`, `pandas.io.formats.style.Styler.hide()`).

In [5]:
df = pd.read_csv("raw_data.csv", skiprows=1)

#Indeks tworzymy z kolumny Eruption Number
df = df.set_index('Eruption Number')
eruptions_data = df[["Volcano Name", "Eruption Category", "VEI", "Start Year", 
                     "Start Month", "Start Day", "End Year", "End Month", "End Day", "Evidence Method (dating)"]]

#Usuwanie z tabeli wiersze zawierające podważane erupcje oraz erupcje bez określonego wskaźnika VEI
eruptions_data = eruptions_data[~eruptions_data["Eruption Category"].str.contains("Discredited", na=False)]
eruptions_data = eruptions_data.dropna(subset=["VEI"])

#Dzielimy kolumne Evidence Method na dwie kolumny (EVidence Type i Evidence Method), 
#następnie zamieniamy wartości NaN odpowiednio na 'Uncertain' i 'Unspecified'.
evidence_split = eruptions_data["Evidence Method (dating)"].str.split(": ", expand=True)
eruptions_data["Evidence Type"] = evidence_split[0].fillna("Uncertain")
eruptions_data["Evidence Method"] = evidence_split[1].fillna("Unspecified")
eruptions_data = eruptions_data.drop(columns=["Evidence Method (dating)"])

#Z nazw w kolumnie Eruption Category usuwamy słowo "Eruption" zostawiając tylko to, co było przed nim
eruptions_data["Eruption Category"] = eruptions_data["Eruption Category"].str.replace(" Eruption", "", regex=False)

#Jeśli początek lub koniec erupcji ma wpisany miesiac lub dzień jako 0 zmieniamy na NaN
date_columns = ["Start Month", "Start Day", "End Year", "End Month", "End Day"]
eruptions_data[date_columns] = eruptions_data[date_columns].replace(0, np.nan)

#Zapisanie do pliku eruption_data.pkl.
eruptions_data.to_pickle("eruption_data.pkl")

eruptions_data_display = eruptions_data.style.format("{:.0f}")
print(eruptions_data.head(10))
print(eruptions_data.tail(10))


                Volcano Name Eruption Category  VEI  Start Year  Start Month  \
Eruption Number                                                                
22486               Cotopaxi         Confirmed  2.0        2022         10.0   
22481                   Taal         Confirmed  1.0        2022         10.0   
22458              Turrialba         Confirmed  1.0        2022          7.0   
22453                 Ulawun         Confirmed  2.0        2022          6.0   
22454                  Raung         Confirmed  2.0        2022          5.0   
22445                   Gaua         Confirmed  1.0        2022          5.0   
22451                 Purace         Confirmed  1.0        2022          3.0   
22430                 Ambrym         Confirmed  1.0        2022          1.0   
22456            Chikurachki         Confirmed  2.0        2022          1.0   
22431                   Wolf         Confirmed  2.0        2022          1.0   

                 Start Day  End Year  E

In [7]:
### Komórka testowa
# Test 1 Kształt tablicy
assert eruptions_data.shape == (8285, 11)
# Test 2 Kolumny
assert len(set(list(eruptions_data.columns)) & set(['Volcano Name', 'Eruption Category', 'VEI', 'Start Year', 'Start Month', 'Start Day',
                                                    'End Year', 'End Month', 'End Day', "Evidence Type", "Evidence Method"])) == 11
# Test 3 Indeks
assert (eruptions_data.index.name == 'Eruption Number') == True
assert eruptions_data.index.values.min() == 10018
assert eruptions_data.index.values.max() == 22488
# Test 4 Zawartość kolumn tekstowych
assert len(eruptions_data["Volcano Name"].unique()) == 714
assert np.all([eruptions_data["Eruption Category"].unique() == ['Confirmed', 'Uncertain']])
assert len(set(list(eruptions_data["Evidence Type"].unique())) & set(['Observations', 'Isotopic', 'Correlation', 'Sidereal', 'Radiogenic', 'Uncertain'])) == 6
assert len(set(list(eruptions_data["Evidence Method"].unique())) & set(['Reported', 'Satellite (infrared)', 'Satellite (visual)',
                                                    'Seismicity', 'Hydrophonic', 'Photo / Video', 'Aviation', 'Uranium-series',
                                                    'Magnetism', '14C (calibrated)', 'Tephrochronology', 'Varve Count', 'Anthropology',
                                                    'Dendrochronology', 'Ice Core', '14C (uncalibrated)', 'Cosmic Ray Exposure', 'Ar/Ar',
                                                    'Fission track', 'K/Ar', 'Thermoluminescence', 'Unspecified'])) == 22
assert eruptions_data["Evidence Method"].isna().sum()==0 or (eruptions_data["Evidence Method"]=='nan').sum()==0 or (eruptions_data["Evidence Method"]=='NaN').sum()==0
# Test 5 Wartości NaN w kolumnach liczbowych
assert eruptions_data["VEI"].isna().sum() == 0
assert eruptions_data["Start Year"].isna().sum() == 0
assert eruptions_data["Start Month"].isna().sum() == 2697
assert eruptions_data["Start Day"].isna().sum() == 3431
assert eruptions_data["End Year"].isna().sum() == 4064
assert eruptions_data["End Month"].isna().sum() == 4300
assert eruptions_data["End Day"].isna().sum() == 4743